In [ ]:
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
from keras import backend as K
import keras.layers as layers
from keras.models import Model, load_model
from keras.engine import Layer
import numpy as np
import sklearn
import imblearn
from imblearn.under_sampling import RandomUnderSampler
import string
# Initialize session
#sess = tf.Session()
#K.set_session(sess)


import pickle
import gzip

In [ ]:
# sess = tf.Session()

# # can't run a gpu on here, would need to build against the sched_mit_sloan_gpu
# #with tf.device('/cpu:0'):
# a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
# b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
# c = tf.matmul(a, b)

# # Creates a session with log_device_placement set to True.
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# print(sess.run(c))

In [ ]:
v6 = pd.read_pickle(
    '/pool001/lraymond/processed_data/pre_analysis_dfs/standardized_linear_classifier_df.p')

In [ ]:
# import and add back in text classification
def load_zipped_pickle(filename):
    # load a zipped compressed pickle file
    with gzip.open(filename, 'rb') as f:
        loaded_object = pickle.load(f)
        return loaded_object
    
index_file = '/pool001/lraymond/patent_data/index_files/sample_pat_nums_all_numerical.gzip'

valid_patents2 = load_zipped_pickle(index_file)

valid_patents2 = valid_patents2.set_index('patent_number')

# here is where I would add in additional pieces of textual data if I wanted

v7 = pd.merge(
    v6, valid_patents2[['patent_abstract', 'patent_title']], left_index=True, right_index=True, how='inner')



print(v6.shape)
print(v7.shape)

v8_text = v7.loc[v7.missing_patent_abstract==0, :]
print(v8_text.shape)

del valid_patents2, v6, v7

In [ ]:
#     dataset = tf.keras.utils.get_file(
#       fname="aclImdb.tar.gz", 
#       origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
#       extract=True)


In [ ]:
class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable = True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

def build(self, input_shape):
        self.elmo = hub.Module(
            'https://tfhub.dev/google/elmo/2', trainable=self.trainable, name="{}_module".format(self.name))
        self.trainable_weights += K.tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)

def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result

def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')

def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

In [ ]:
# Function to build model
def build_model(): 
    input_text = layers.Input(shape=(1,), dtype="string")
    embedding = ElmoEmbeddingLayer()(input_text)
    dense = layers.Dense(256, activation='relu')(embedding)
    pred = layers.Dense(1, activation='sigmoid')(dense)
    model = Model(inputs=[input_text], outputs=pred)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    return model



In [ ]:
string.punctuation

In [ ]:
def clean_abstract(abstract_str):
    # remove punctuation from each word
    table = str.maketrans('', '', string.punctuation)
    try:
        str2 = abstract_str.strip()
        return ' '.join(w.translate(table) for w in str2.split() if w.isalpha())
    except AttributeError as e:
        print(e, abstract_str)
        return np.nan

In [ ]:
# need to remove punctuation from abstracts

v8_text['patent_abstract'] = v8_text['patent_abstract'].apply(clean_abstract)

In [ ]:
v8_text.loc[pd.isnull(v8_text.patent_abstract), 'missing_patent_abstract'] = 1

In [ ]:
v8_text.head()

In [ ]:
# input data should be a dataframe of sentences and target value
text_cols = ['patent_abstract']
Y_col = ['10_year_cites_top1']
model_df = v8_text.loc[v8_text['missing_patent_abstract']==0, text_cols + Y_col].copy()

# 1:1 ratio of top 1 and not
rus = imblearn.under_sampling.RandomUnderSampler(sampling_strategy=1, return_indices=True, random_state=0)

X = model_df[text_cols].as_matrix()
Y = model_df[Y_col].as_matrix()
# (X[ind]==X_resampled).all() should be True
X_resampled, y_resampled, ind = rus.fit_resample(X, Y)
print(X_resampled.shape, y_resampled.shape, y_resampled.mean())
    
X_train, X_test, y_train, y_test, pat_nums_train, pat_nums_test = sklearn.model_selection.train_test_split(
    X_resampled, y_resampled, ind, random_state=0, shuffle=True)

In [ ]:
train_text = model_df['patent_abstract'].tolist()
train_text = np.array(train_text, dtype=object)[:, np.newaxis]

In [ ]:
train_text[0][0]

In [ ]:
# Build and fit
model = build_model()
model.fit(X_train, 
          y_train,
          validation_data=(X_test, y_test),
          epochs=5,
          batch_size=128)


In [ ]:
model.save('ElmoModel.h5')
pre_save_preds = model.predict(test_text[0:100]) # predictions before we clear and reload model

# Clear and load model
model = None
model = build_model()
model.load_weights('ElmoModel.h5')

In [ ]:
# restart and reload so can use pre trained elmo embeddings, weakly train classifier on patent abstract and title
# then estimate model accuracy.
# save embeddings in numpy and then load them into tensor scratch for model building
# then, write a script that generates output for a variety of combos of NNs and SVMs with and without text classification
# next step is to try to train embeddings from scratch
# then switch output to a different function
# write down a list of questions to answer for meeting on Friday

In [ ]:
# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# print(sess.run(c))

In [ ]:
elmo = hub.Module('https://tfhub.dev/google/elmo/2', trainable=False)
embeddings = elmo(
    ["the cat is on the mat", "what are you doing in evening"],signature="default",
    as_dict=True)["elmo"]
with tf.Session() as session:
    tf.initialize_all_variables.run()
    tf.tables_initializer().run()
    #session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embeddings)

In [3]:
message_embeddings

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/software/sloan/local/lib/py36/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-37e872c55f81>", line 1, in <module>
    message_embeddings
NameError: name 'message_embeddings' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/software/sloan/local/lib/py36/IPython/core/interactiveshell.py", line 1863, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/software/sloan/local/lib/py36/IPython/core/ultratb.py", line 1095, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/home/software/sloan/local/lib/py36/IPython/core/ultratb.py", lin

NameError: name 'message_embeddings' is not defined